
Deposition of integrative models<a id="mainpage"></a>
========================

**Table of contents**

 - [Introduction](#introduction)
 - [Why PDB-IHM?](#whypdbihm)
 - [Why mmCIF?](#whymmcif)
 - [Study versus single model](#studyvmodel)
 - [File format](#fileformat)
 - [Basic usage of ProtocolOutput](#basicusage)
 - [mmCIF file format](#mmcif)
 - [Linking to other data](#linking)
 - [Annotation of input files](#annotation)
 - [Polishing the deposition](#polishing)
  - [Cross-linker type](#xltype)
  - [Correct number of output models](#fixnummodel)
  - [Add UniProt sequence information](#uniprot)
  - [Add model coordinates](#addcoords)
  - [Replace local links with DOIs](#adddois)
 - [Output](#output)
 - [Visualization](#visualization)
  - [ChimeraX](#chimerax)
  - [Web browser](#browser)
  - [Plain text](#plain)
  - [As Python objects](#python)
 - [Further reading](#further)


# Introduction<a id="introduction"></a>

In this tutorial we will introduce the procedure used to deposit integrative modeling studies in the [PDB-IHM](https://pdb-ihm.org/) database in mmCIF format.

We will demonstrate the procedure using [IMP](https://integrativemodeling.org/) and its PMI module, but the database will accept integrative models from any software, as long as they are compliant mmCIF files (e.g. there are several HADDOCK and Rosetta models already in PDB-IHM).

# Why PDB-IHM?<a id="whypdbihm"></a>

PDB-IHM is a database run by the wwPDB. It is specifically for the deposition of *integrative* models, i.e. models generated using more than one source of experimental data. (Models that use only one experiment generally go in PDB; models that use no experimental information - theoretical models - go in [ModelArchive](https://www.modelarchive.org/)).

# Why mmCIF?<a id="whymmcif"></a>

wwPDB already uses the mmCIF file format for X-ray structures, and has a formal data model ([PDBx](http://mmcif.wwpdb.org/)) to describe these structures. The format is *extensible*; extension dictionaries exist to describe NMR, SAS, EM data, etc. For integrative models, we use PDBx plus an "integrative/hybrid methods" (IHM) [extension dictionary](http://mmcif.wwpdb.org/dictionaries/mmcif_ihm.dic/Index/). This supports coarse-grained structures, multiple input experimental data sources, multiple states, multiple scales, and ensembles related by time or other order.

# Study versus single model<a id="studyvmodel"></a>

A frequently asked question is whether mmCIF files can be generated directly from an [IMP::Model](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1Model.html) or an
[RMF file](https://integrativemodeling.org/rmf/). This generally isn't possible because RMF and [IMP::Model](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1Model.html) are designed to store one or more output models, where each model is a set of coordinates for a single conformation of the system being studied. A deposition, on the other hand, aims to cover a complete modeling study, and should capture the entire ensemble of output models, all of the input data needed to reproduce the modeling, and quality metrics such as the precision of the ensemble and the degree to which it fits the output data. A deposition is also designed to
be visualized, and so may contain additional data not used in the modeling itself, such as preset colors or views to match figures in the publication or highlight regions of interest, and more human-descriptive names for parts
of the system. Thus, deposition is largely a data-gathering exercise, and benefits from a modeling study being tidy and well organized (for example, by storing it in a [GitHub](https://github.com) repository) so that data is easy to find and track to its source.

# File format<a id="fileformat"></a>

The file format used by PDB for deposition of integrative modeling studies is the same [mmCIF format](http://mmcif.wwpdb.org/) that is used for crystal structures, with extensions (the [Integrative/Hybrid Methods (IHM) dictionary](http://mmcif.wwpdb.org/dictionaries/mmcif_ihm.dic/Index/)) to support coarse-grained structures, multiple input experimental data sources, multiple states, multiple scales, and ensembles related by time or other order. Currently, models that are compliant with the IHM dictionary can be deposited in the [PDB-IHM](https://pdb-ihm.org/) database.

mmCIF is a text format with a well-defined syntax, so in principle files could be generated by hand or with simple scripts. However, it is generally easier to use the existing [python-ihm](https://github.com/ihmwg/python-ihm)
library. This stores the same data as in an mmCIF file, but represents it as a set of Python classes, so it is easier to manipulate. IMP includes a copy of the python-ihm library, and uses it internally to read and write mmCIF files.

There are two main approaches to generating an mmCIF file for deposition:

 1. Use the [python-ihm](https://github.com/ihmwg/python-ihm) library directly, by writing a Python script that reads in output models and input data, adds annotations, and writes out an mmCIF file.
 2. Use the [ProtocolOutput](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1pmi_1_1mmcif_1_1ProtocolOutput.html) class to automatically capture an entire [IMP::pmi](https://integrativemodeling.org/2.23.0/doc/ref/namespaceIMP_1_1pmi.html) modeling protocol.

The first approach offers the greatest flexibility for custom protocols or for modeling outside of IMP itself, but does require everything to be done manually. For a good example of this approach, see the [modeling of Nup133](https://github.com/integrativemodeling/nup133/tree/main/outputs_foxs_ensemble_new/pdb-dev). (One example of the tradeoff between flexibility and manual coding: the cross-links used in the study are stored in a [plain text file](https://github.com/integrativemodeling/nup133/blob/main/Crosslinks/DSS_EDC_crosslinks.txt) so a [custom Python class](https://github.com/integrativemodeling/nup133/blob/main/outputs_foxs_ensemble_new/pdb-dev/xlink.py) had to be written to parse them.)

The second approach will be explored in this tutorial.

# Basic usage of ProtocolOutput<a id="basicusage"></a>

[ProtocolOutput](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1pmi_1_1mmcif_1_1ProtocolOutput.html) is designed to be attached to a top-level PMI object (usually [IMP::pmi::topology::System](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1pmi_1_1topology_1_1System.html)). Then, as the script is run, it will capture all of the information about the modeling
study, in an [ihm.System](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System) object. Additional information not in the modeling script itself, such as the
resulting publication, can be added using the [python-ihm API](https://python-ihm.readthedocs.io/en/latest/usage.html).


We will demonstrate ProtocolOutput by using it to make a deposition of the RNA Pol II stalk modeling, as covered in the [introductory PMI tutorial](https://integrativemodeling.org/tutorials/rnapolii_stalk/). First, download the files for this tutorial by using the "Clone or download" link at the [tutorial's GitHub page](https://github.com/salilab/imp_deposition_tutorial/).

We will then modify the `modeling.py` script from the introductory tutorial, calling it [deposition.py](https://github.com/salilab/imp_deposition_tutorial/blob/main/rnapolii/modeling/deposition.py) in the `rnapolii/modeling` subdirectory, to capture the study as mmCIF.

The first modification to `deposition.py` is to import the PMI mmCIF and python-ihm Python modules:

In [1]:
# Imports needed to use ProtocolOutput
import IMP.pmi.mmcif
import ihm
import ihm.location
import ihm.reference
import ihm.model

The script then runs unchanged, as in the original PMI tutorial, to set up the topology:

In [2]:
from pathlib import Path
import time
import sys
import math
import pandas as pd
import argparse

import IMP.pmi.restraints.crosslinking
import IMP.pmi.restraints.stereochemistry
import IMP.pmi.restraints.em
import IMP.pmi.restraints.basic
import IMP.bayesianem
import IMP.bayesianem.restraint
import IMP.algebra
import IMP.pmi.restraints.stereochemistry
import IMP.pmi.restraints.basic
import IMP.pmi.io.crosslink
import IMP.pmi.restraints
import IMP.pmi.restraints.crosslinking
import IMP.pmi.restraints.stereochemistry
import ihm.cross_linkers
import IMP
import IMP.pmi
import IMP.pmi.topology
import IMP.pmi.dof
import IMP.pmi.macros

In [3]:


#---------------------------
# Define Input Files
#---------------------------
# datadirectory = "../data/"
# topology_file = datadirectory+"topology.txt"
# target_gmm_file = datadirectory+'emd_1883.map.mrc.gmm.50.txt'

# # Initialize model
# m = IMP.Model()

# # Read in the topology file.
# # Specify the directory wheere the PDB files, fasta files and GMM files are
# topology = IMP.pmi.topology.TopologyReader(topology_file,
#                                   pdb_dir=datadirectory,
#                                   fasta_dir=datadirectory,
#                                   gmm_dir=datadirectory)

In [4]:
glob_data_dir = Path(Path.home(), "Documents/mtorc2/data")
fasta_file = Path(glob_data_dir, "fasta/mtorc2.fasta")

param_df = pd.read_csv(Path(glob_data_dir, "params/130.csv"))

print(param_df.head())


     name subunit   color  rb_start  rb_end  \
0    MTOR    MTOR    blue         1    2549   
1  RICTOR  RICTOR   green         1    1708   
2   MLST8   MLST8  yellow         1     326   
3   MSIN1   MSIN1     red         1     154   
4    CRIM   MSIN1     red       155     277   

                                            pdb_file  \
0  /wynton/home/sali/mhancock/mtorc2/data/em/mode...   
1  /wynton/home/sali/mhancock/mtorc2/data/em/mode...   
2  /wynton/home/sali/mhancock/mtorc2/data/em/mode...   
3  /wynton/home/sali/mhancock/mtorc2/data/em/mode...   
4  /wynton/home/sali/mhancock/mtorc2/data/em/mode...   

                                           em_prefix  res_per_comp  start_pdb  \
0                                                NaN             0          1   
1                                                NaN             0          1   
2                                                NaN             0          1   
3                                                NaN    

In [5]:
## ok in the params_df there is a pdb_file field. i want to change the beginning of each of those pdb_file entries to be /Users/matthew/Documents instead of /wynton/home/sali/mhancock
old_prefix = r"^/wynton/home/sali/mhancock"
new_prefix = "/Users/matthew/Documents"

mask = param_df["pdb_file"].notna()
param_df.loc[mask, "pdb_file"] = (
    param_df.loc[mask, "pdb_file"].str.replace(old_prefix, new_prefix, regex=True)
)

## change em prefix too
mask = param_df["em_prefix"].notna()
param_df.loc[mask, "em_prefix"] = (
    param_df.loc[mask, "em_prefix"].str.replace(old_prefix, new_prefix, regex=True)
)

# Optional: preview changes
print(param_df[["name", "pdb_file", "em_prefix"]].head())

     name                                           pdb_file  \
0    MTOR  /Users/matthew/Documents/mtorc2/data/em/models...   
1  RICTOR  /Users/matthew/Documents/mtorc2/data/em/models...   
2   MLST8  /Users/matthew/Documents/mtorc2/data/em/models...   
3   MSIN1  /Users/matthew/Documents/mtorc2/data/em/models...   
4    CRIM  /Users/matthew/Documents/mtorc2/data/em/models...   

                                           em_prefix  
0                                                NaN  
1                                                NaN  
2                                                NaN  
3                                                NaN  
4  /Users/matthew/Documents/mtorc2/data/em/comps/...  


In [6]:
m = IMP.Model()
s = IMP.pmi.topology.System(m)
seqs = IMP.pmi.topology.Sequences(fasta_fn=str(fasta_file))
st = s.create_state()

In [7]:
# Record the modeling protocol to an mmCIF file
po = IMP.pmi.mmcif.ProtocolOutput()
s.add_protocol_output(po)
s.title = "mTORC2-AKT1"

In [8]:


clones = dict()
mols = dict()
subunits = ["MTOR", "RICTOR", "MLST8", "MSIN1", "AKT1"]
for subunit in subunits:
    subunit_row_ids = list(param_df[param_df["subunit"] == subunit].index)
    color = param_df.iloc[subunit_row_ids[0], param_df.columns.get_loc("color")]
    chain = param_df.iloc[subunit_row_ids[0], param_df.columns.get_loc("model_chain")]

    print(subunit, chain)
    mol = st.create_molecule(
        name=subunit,
        sequence=seqs[subunit],
        chain_id=chain
    )
    mols[subunit] = mol

    print(subunit_row_ids)
    for row_id in subunit_row_ids:
        name = param_df.iloc[row_id]["name"]
        pdb_file = Path(param_df.iloc[row_id]["pdb_file"])
        pdb_chain = param_df.iloc[row_id]["pdb_chain"]
        start_pdb = param_df.iloc[row_id]["start_pdb"]
        end_pdb = param_df.iloc[row_id]["end_pdb"]
        offset = int(param_df.iloc[row_id]["offset"])
        em_prefix = param_df.iloc[row_id]["em_prefix"]
        res_per_comp = int(param_df.iloc[row_id]["res_per_comp"])
        print(name, pdb_file, pdb_chain, start_pdb, end_pdb, offset, em_prefix)

        atom_res = mol.add_structure(
            pdb_fn=str(pdb_file),
            chain_id=pdb_chain,
            soft_check=True,
            res_range=(start_pdb, end_pdb),
            offset=offset,
            model_num=1
        )

        if type(em_prefix) == str:
            mol.add_representation(
                residues=atom_res,
                density_residues_per_component=res_per_comp,
                density_prefix=str(em_prefix),
                density_force_compute=False,
                resolutions=[1,10],
                color=color
            )
        else:
            mol.add_representation(
                residues=atom_res,
                resolutions=[1,10],
                color=color
            )

    if True:
        if subunit == "AKT1":
            print(mol.get_non_atomic_residues())
            flex_res = list(mol.get_non_atomic_residues())[:22]
        else:
            flex_res = list(mol.get_non_atomic_residues())

        mol.add_representation(
            flex_res,
            resolutions=[10],
            color=color,
            setup_particles_as_densities=False
        )

    if False and subunit == "AKT1":
        # Add any residues past >425 that aren't already represented.
        # Have to hard code these numbers in.
        akt_tail = mols["AKT1"].residue_range(
            a=str(426),
            b=str(480)
        )

        print(len(akt_tail))

        akt_tail_flex = [res for res in akt_tail if res not in mol.get_atomic_residues()]

        print(len(akt_tail_flex))

        mol.add_representation(
            akt_tail_flex,
            resolutions=[1],
            color=color
        )

    print(subunit, len(mol.get_residues()))
    print(subunit, len(mol.get_atomic_residues()))
    print(subunit, len(mol.get_non_atomic_residues()))

    if True:
        print("CREATING CLONE")
        clone = mol.create_clone(
            chain_id=chr(ord(chain) + 5)
        )
        clones[subunit] = clone



MTOR A
[0]
MTOR /Users/matthew/Documents/mtorc2/data/em/models/mTORC2-Akt_v1113_Dimer.pdb F 1 2549 0 nan
MTOR 2549
MTOR 2184
MTOR 365
CREATING CLONE


/Users/matthew/miniforge3/envs/imp_221/lib/python3.12/site-packages/IMP/pmi/topology/__init__.py:1317: StructureWarning: Inconsistency between FASTA sequence and PDB sequence. FASTA type 631 "THR" and PDB type "ILE"
  warnings.warn(
/Users/matthew/miniforge3/envs/imp_221/lib/python3.12/site-packages/IMP/pmi/topology/__init__.py:1317: StructureWarning: Inconsistency between FASTA sequence and PDB sequence. FASTA type 637 "ILE" and PDB type "ALA"
  warnings.warn(


RICTOR B
[1]
RICTOR /Users/matthew/Documents/mtorc2/data/em/models/mTORC2-Akt_v1113_Dimer.pdb H 1 1708 0 nan
begin read_pdb:
  WARNING  Could not determine CHARMM atom type for atom "HET:ZN  " in residue #1 "ZN"
end read_pdb
RICTOR 1708
RICTOR 1171
RICTOR 537
CREATING CLONE
MLST8 C
[2]
MLST8 /Users/matthew/Documents/mtorc2/data/em/models/mTORC2-Akt_v1113_Dimer.pdb G 1 326 0 nan
MLST8 326
MLST8 317
MLST8 9
CREATING CLONE
MSIN1 D
[3, 4, 5, 6]
MSIN1 /Users/matthew/Documents/mtorc2/data/em/models/mTORC2-Akt_v1113_Dimer.pdb I 1 154 0 nan
CRIM /Users/matthew/Documents/mtorc2/data/em/models/mTORC2-Akt_v1113_Dimer.pdb I 155 267 0 /Users/matthew/Documents/mtorc2/data/em/comps/1131/10/CRIM
RBD /Users/matthew/Documents/mtorc2/data/pdb/7lc1.pdb B 278 353 0 nan
MSIN1PH /Users/matthew/Documents/mtorc2/data/pdb/7lc1.pdb B 383 481 0 nan
MSIN1 522
MSIN1 403
MSIN1 119
CREATING CLONE
AKT1 E
[7, 8, 9]
AKT1PH /Users/matthew/Documents/mtorc2/data/pdb/akt1_af.pdb H 1 123 0 nan
KINASEN /Users/matthew/Document

/Users/matthew/miniforge3/envs/imp_221/lib/python3.12/site-packages/IMP/pmi/topology/__init__.py:1317: StructureWarning: Inconsistency between FASTA sequence and PDB sequence. FASTA type 74 "SER" and PDB type "THR"
  warnings.warn(


Next, as soon as the top-level [IMP::pmi::topology::System](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1pmi_1_1topology_1_1System.html) object is created, we attach a ProtocolOutput object (BuildSystem contains a `system` member):

In [9]:
# # Use the BuildSystem macro to build states from the topology file
# bs = IMP.pmi.macros.BuildSystem(m)

# # Record the modeling protocol to an mmCIF file
# po = IMP.pmi.mmcif.ProtocolOutput()
# bs.system.add_protocol_output(po)
# po.system.title = "mTORC2-AKT1"
# # Add publication
# po.system.citations.append(ihm.Citation.from_pubmed_id(25161197))

In [10]:
# Use the BuildSystem macro to build states from the topology file
root_hier = s.build()
dof = IMP.pmi.dof.DegreesOfFreedom(m)

# # Record the modeling protocol to an mmCIF file
# po = IMP.pmi.mmcif.ProtocolOutput()
# s.add_protocol_output(po)
# s.title = "mTORC2-AKT1"

done building "MTOR" Chain F Copy: 1
done building "MTOR" Chain A Copy: 0
done building "RICTOR" Chain G Copy: 1
begin read_pdb:
  WARNING  Could not determine CHARMM atom type for atom "HET:ZN  " in residue #1 "ZN"
end read_pdb
begin read_pdb:
  WARNING  Could not determine CHARMM atom type for atom "HET:ZN  " in residue #1 "ZN"
end read_pdb
begin read_pdb:
  WARNING  Could not determine CHARMM atom type for atom "HET:ZN  " in residue #1 "ZN"
end read_pdb
begin read_pdb:
  WARNING  Could not determine CHARMM atom type for atom "HET:ZN  " in residue #1 "ZN"
end read_pdb
begin read_pdb:
  WARNING  Could not determine CHARMM atom type for atom "HET:ZN  " in residue #1 "ZN"
end read_pdb
done building "RICTOR" Chain B Copy: 0
begin read_pdb:
  WARNING  Could not determine CHARMM atom type for atom "HET:ZN  " in residue #1 "ZN"
end read_pdb
begin read_pdb:
  WARNING  Could not determine CHARMM atom type for atom "HET:ZN  " in residue #1 "ZN"
end read_pdb
begin read_pdb:
  WARNING  Could not

/Users/matthew/miniforge3/envs/imp_221/lib/python3.12/site-packages/IMP/pmi/topology/__init__.py:865: StructureWarning: Residues without representation in molecule AKT1: 210-212, 230-233, 426-480
  warnings.warn(


begin read_pdb:
  WARNING  Could not determine CHARMM atom type for atom "HET: N  " in residue #473 "T1C"
  WARNING  Could not determine CHARMM atom type for atom "HET: CA " in residue #473 "T1C"
  WARNING  Could not determine CHARMM atom type for atom "HET: C  " in residue #473 "T1C"
  WARNING  Could not determine CHARMM atom type for atom "HET: O  " in residue #473 "T1C"
  WARNING  Could not determine CHARMM atom type for atom "HET: CB " in residue #473 "T1C"
  WARNING  Could not determine CHARMM atom type for atom "HET: SG " in residue #473 "T1C"
  WARNING  Could not determine CHARMM atom type for atom "HET: C05" in residue #473 "T1C"
  WARNING  Could not determine CHARMM atom type for atom "HET: C06" in residue #473 "T1C"
  WARNING  Could not determine CHARMM atom type for atom "HET: C07" in residue #473 "T1C"
  WARNING  Could not determine CHARMM atom type for atom "HET: C10" in residue #473 "T1C"
  WARNING  Could not determine CHARMM atom type for atom "HET: C11" in residue #473 

In [11]:
s.get_states()[0].get_molecules()

OrderedDict([('MTOR', [System.State_0.MTOR.0, System.State_0.MTOR.1]),
             ('RICTOR', [System.State_0.RICTOR.0, System.State_0.RICTOR.1]),
             ('MLST8', [System.State_0.MLST8.0, System.State_0.MLST8.1]),
             ('MSIN1', [System.State_0.MSIN1.0, System.State_0.MSIN1.1]),
             ('AKT1', [System.State_0.AKT1.0, System.State_0.AKT1.1])])

In [12]:
po.system.asym_units

Note that the ProtocolOutput object `po` simply wraps an [ihm.System](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System) object as `po.system`. We can then customize the [ihm.System](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System) by setting a human-readable title and adding a citation (here we use
[ihm.Citation.from_pubmed_id](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.Citation.from_pubmed_id), which looks up a citation by PubMed ID - this particular PubMed ID is
actually for the previously-published [modeling of the Nup84 complex](https://salilab.org/nup84/)).

Now the original script proceeds as before, setting up the representation and restraints:

In [13]:
rbs = dict()
for subunit in subunits:
    rbs[subunit] = list()
    copies = list()
    copies.append(mols[subunit])
    if True:
        copies.append(clones[subunit])

    subunit_row_ids = list(param_df[param_df["subunit"] == subunit].index)
    for row_id in subunit_row_ids:
        rb_start = param_df.iloc[row_id]["rb_start"]
        rb_end = param_df.iloc[row_id]["rb_end"]
        rb_trans = param_df.iloc[row_id]["rb_trans"]
        rb_rot = param_df.iloc[row_id]["rb_rot"]
        non_rigid_trans = param_df.iloc[row_id]["flex_trans"]

        for copy in copies:
            rigid_parts = copy.residue_range(
                a=str(rb_start),
                b=str(rb_end)
            )

            non_rigid_parts = [res for res in copy.get_non_atomic_residues() if res in rigid_parts]

            rb_movers, rb = dof.create_rigid_body(
                rigid_parts=rigid_parts,
                nonrigid_parts=non_rigid_parts,
                max_trans=rb_trans,
                max_rot=rb_rot,
                nonrigid_max_trans=non_rigid_trans
            )

            # if copy == mol:
            rbs[subunit].append(rb)

In [14]:
# CONNECTIVITY
output_objects = []
for subunit in subunits:
    cr = IMP.pmi.restraints.stereochemistry.ConnectivityRestraint(
        objects=mols[subunit],
        resolution=1
    )
    cr.add_to_model()
    output_objects.append(cr)

print("AKT tail restraints")
# for i in range(425,480):
#     conn_r = IMP.pmi.restraints.basic.DistanceRestraint(
#         root_hier=root_hier,
#         tuple_selection1=(i,i,"AKT1",0),
#         tuple_selection2=(i+1,i+1,"AKT1",0),
#         distancemin=3.5,
#         distancemax=3.5,
#         weight=100
#     )
#     conn_r.add_to_model()
#     output_objects.append(conn_r)

# EXCLUDED VOLUME
ev_objects = list()
for subunit in subunits:
    ev_objects.append(mols[subunit])

    ev_objects.append(clones[subunit])

ev_r = IMP.pmi.restraints.stereochemistry.ExcludedVolumeSphere(
    included_objects=ev_objects,
    resolution=10
)
ev_r.add_to_model()
output_objects.append(ev_r)

# SYMMETRY
center = IMP.algebra.Vector3D([166.9824, 166.9824, 166.9824])
rot = IMP.algebra.get_rotation_about_axis([0.0,0.0,1.0], math.pi)
transform = IMP.algebra.get_rotation_about_point(center, rot)

for subunit in subunits:
    dof.constrain_symmetry(
        references=mols[subunit],
        clones=clones[subunit],
        transform=transform
    )

m.update()


Adding sequence connectivity restraint between 1-10_bead  and  11-13_bead of distance 3.6
Adding sequence connectivity restraint between 11-13_bead  and  Residue_14 of distance 3.6
Adding sequence connectivity restraint between Residue_22  and  23-32_bead of distance 3.6
Adding sequence connectivity restraint between 23-32_bead  and  33-42_bead of distance 3.6
Adding sequence connectivity restraint between 33-42_bead  and  43-45_bead of distance 3.6
Adding sequence connectivity restraint between 43-45_bead  and  Residue_46 of distance 3.6
Adding sequence connectivity restraint between Residue_53  and  54-63_bead of distance 3.6
Adding sequence connectivity restraint between 54-63_bead  and  64-73_bead of distance 3.6
Adding sequence connectivity restraint between 64-73_bead  and  74-79_bead of distance 3.6
Adding sequence connectivity restraint between 74-79_bead  and  Residue_80 of distance 3.6
Adding sequence connectivity restraint between Residue_93  and  94-103_bead of distance 3.6

In [15]:
# XLS
xl_dir = Path(glob_data_dir, "xlms/csvs")
xldbkwc = IMP.pmi.io.crosslink.CrossLinkDataBaseKeywordsConverter()
xldbkwc.set_protein1_key("prot1")
xldbkwc.set_protein2_key("prot2")
xldbkwc.set_residue1_key("res1")
xldbkwc.set_residue2_key("res2")

xldb = IMP.pmi.io.crosslink.CrossLinkDataBase(xldbkwc)

xls = "dss:1:DSS,edc:1:EDC"
for xl in xls.split(","):
    xl, xl_w, xl_type = xl.split(":")
    xl_file = Path(xl_dir, xl+".csv")
    xldb.create_set_from_file(str(xl_file))

    if xl_type == "DSS":
        linker = ihm.cross_linkers.dss
        length = 35
    elif xl_type == "EDC":
        linker = ihm.cross_linkers.edc
        length = 16

    xl_r = IMP.pmi.restraints.crosslinking.CrossLinkingMassSpectrometryRestraint(
        root_hier=root_hier,
        database=xldb,
        length=length,
        slope=0.01,
        resolution=1.,
        label=xl_type,
        linker=linker,
        weight=float(xl_w)
    )

    xl_r.add_to_model()
    output_objects.append(xl_r)


gathering copies
defaultdict(<class 'int'>, {'AKT1': 1, 'MTOR': 1, 'MSIN1': 1, 'RICTOR': 1, 'MLST8': 1})
done pmi2 prelims
generating a new cross-link restraint
--------------
CrossLinkingMassSpectrometryRestraint: generating cross-link restraint between
CrossLinkingMassSpectrometryRestraint: residue 183 of chain AKT1.0 and residue 182 of chain AKT1.0
CrossLinkingMassSpectrometryRestraint: with sigma1 SIGMA sigma2 SIGMA psi PSI
CrossLinkingMassSpectrometryRestraint: between particles Residue_183 and Residue_182

--------------
CrossLinkingMassSpectrometryRestraint: generating cross-link restraint between
CrossLinkingMassSpectrometryRestraint: residue 183 of chain AKT1.1 and residue 182 of chain AKT1.0
CrossLinkingMassSpectrometryRestraint: with sigma1 SIGMA sigma2 SIGMA psi PSI
CrossLinkingMassSpectrometryRestraint: between particles Residue_183 and Residue_182

--------------
CrossLinkingMassSpectrometryRestraint: generating cross-link restraint between
CrossLinkingMassSpectrometryRes

/Users/matthew/miniforge3/envs/imp_221/lib/python3.12/site-packages/IMP/pmi/restraints/crosslinking.py:214: StructureWarning: CrossLinkingMassSpectrometryRestraint: residue 426 of chain AKT1.0 is not there
  warnings.warn(
/Users/matthew/miniforge3/envs/imp_221/lib/python3.12/site-packages/IMP/pmi/restraints/crosslinking.py:214: StructureWarning: CrossLinkingMassSpectrometryRestraint: residue 426 of chain AKT1.1 is not there
  warnings.warn(


--------------
CrossLinkingMassSpectrometryRestraint: generating cross-link restraint between
CrossLinkingMassSpectrometryRestraint: residue 1277 of chain MTOR.1 and residue 230 of chain MTOR.0
CrossLinkingMassSpectrometryRestraint: with sigma1 SIGMA sigma2 SIGMA psi PSI
CrossLinkingMassSpectrometryRestraint: between particles Residue_1277 and 227-232_bead

--------------
CrossLinkingMassSpectrometryRestraint: generating cross-link restraint between
CrossLinkingMassSpectrometryRestraint: residue 1277 of chain MTOR.0 and residue 230 of chain MTOR.1
CrossLinkingMassSpectrometryRestraint: with sigma1 SIGMA sigma2 SIGMA psi PSI
CrossLinkingMassSpectrometryRestraint: between particles Residue_1277 and 227-232_bead

--------------
CrossLinkingMassSpectrometryRestraint: generating cross-link restraint between
CrossLinkingMassSpectrometryRestraint: residue 1277 of chain MTOR.1 and residue 230 of chain MTOR.1
CrossLinkingMassSpectrometryRestraint: with sigma1 SIGMA sigma2 SIGMA psi PSI
CrossLin

In [16]:
# # Each state can be specified by a topology file.
# bs.add_state(topology)

# # Build the system representation and degrees of freedom
# root_hier, dof = bs.execute_macro(max_rb_trans=4.0,
#                                   max_rb_rot=0.3,
#                                   max_bead_trans=4.0,
#                                   max_srb_trans=4.0,
#                                   max_srb_rot=0.3)

# # Fix all rigid bodies but not Rpb4 and Rpb7 (the stalk)
# # First select and gather all particles to fix.
# fixed_particles=[]
# for prot in ["Rpb1","Rpb2","Rpb3","Rpb5","Rpb6","Rpb8","Rpb9","Rpb10","Rpb11","Rpb12"]:
#     fixed_particles+=IMP.atom.Selection(root_hier,molecule=prot).get_selected_particles()

# # Fix the Corresponding Rigid movers and Super Rigid Body movers using dof
# # The flexible beads will still be flexible (fixed_beads is an empty list)!
# fixed_beads,fixed_rbs=dof.disable_movers(fixed_particles,
#                                          [IMP.core.RigidBodyMover,
#                                           IMP.pmi.TransformMover])

# # Randomize the initial configuration before sampling, of only the molecules
# # we are interested in (Rpb4 and Rpb7)
# IMP.pmi.tools.shuffle_configuration(root_hier,
#                                     excluded_rigid_bodies=fixed_rbs,
#                                     max_translation=50,
#                                     verbose=False,
#                                     cutoff=5.0,
#                                     niterations=100)

# outputobjects = [] # reporter objects (for stat files)

# #-----------------------------------
# # Define Scoring Function Components
# #-----------------------------------

# # Here we are defining a number of restraints on our system.
# #  For all of them we call add_to_model() so they are incorporated into scoring
# #  We also add them to the outputobjects list, so they are reported in stat files

# # Connectivity keeps things connected along the backbone (ignores if inside
# # same rigid body)
# mols = IMP.pmi.tools.get_molecules(root_hier)
# for mol in mols:
#     molname=mol.get_name()
#     IMP.pmi.tools.display_bonds(mol)
#     cr = IMP.pmi.restraints.stereochemistry.ConnectivityRestraint(
#         mol,scale=2.0, label=molname)
#     cr.add_to_model()
#     outputobjects.append(cr)

# # Excluded Volume Restraint
# #  To speed up this expensive restraint, we operate it at resolution 20
# ev = IMP.pmi.restraints.stereochemistry.ExcludedVolumeSphere(
#                                          included_objects=root_hier,
#                                          resolution=10)
# ev.add_to_model()
# outputobjects.append(ev)


# # Crosslinks - dataset 1
# #  To use this restraint we have to first define the data format
# #  Here assuming that it's a CSV file with column names that may need to change
# #  Other options include the linker length and the slope (for nudging components together)
# xldbkwc = IMP.pmi.io.crosslink.CrossLinkDataBaseKeywordsConverter()
# xldbkwc.set_protein1_key("pep1.accession")
# xldbkwc.set_protein2_key("pep2.accession")
# xldbkwc.set_residue1_key("pep1.xlinked_aa")
# xldbkwc.set_residue2_key("pep2.xlinked_aa")

# xl1db = IMP.pmi.io.crosslink.CrossLinkDataBase(xldbkwc)
# xl1db.create_set_from_file(datadirectory+'polii_xlinks.csv')

# xl1 = IMP.pmi.restraints.crosslinking.CrossLinkingMassSpectrometryRestraint(
#                                    root_hier=root_hier,
#                                    database=xl1db,
#                                    length=21.0,
#                                    slope=0.01,
#                                    resolution=1.0,
#                                    label="Trnka",
#                                    linker=ihm.cross_linkers.dss,
#                                    weight=1.)

# xl1.add_to_model()             # crosslink must be added to the model
# outputobjects.append(xl1)


# # Crosslinks - dataset 2
# #  We can easily add a second set of crosslinks.
# #  These have a different format and label, but other settings are the same
# xldbkwc = IMP.pmi.io.crosslink.CrossLinkDataBaseKeywordsConverter()
# xldbkwc.set_protein1_key("prot1")
# xldbkwc.set_protein2_key("prot2")
# xldbkwc.set_residue1_key("res1")
# xldbkwc.set_residue2_key("res2")

# xl2db = IMP.pmi.io.crosslink.CrossLinkDataBase(xldbkwc)
# xl2db.create_set_from_file(datadirectory+'polii_juri.csv')

# xl2 = IMP.pmi.restraints.crosslinking.CrossLinkingMassSpectrometryRestraint(
#                                    root_hier=root_hier,
#                                    database=xl2db,
#                                    length=21.0,
#                                    slope=0.01,
#                                    resolution=1.0,
#                                    label="Chen",
#                                    linker=ihm.cross_linkers.bs3,
#                                    weight=1.)
# xl2.add_to_model()
# outputobjects.append(xl2)


# # Electron Microscopy Restraint
# #  The GaussianEMRestraint uses a density overlap function to compare model to data
# #   First the EM map is approximated with a Gaussian Mixture Model (done separately)
# #   Second, the components of the model are represented with Gaussians (forming the model GMM)
# #   Other options: scale_to_target_mass ensures the total mass of model and map are identical
# #                  slope: nudge model closer to map when far away
# #                  weight: experimental, needed because the EM restraint is quasi-Bayesian
# em_components = IMP.pmi.tools.get_densities(root_hier)

# gemt = IMP.pmi.restraints.em.GaussianEMRestraint(em_components,
#                                                  target_gmm_file,
#                                                  scale_target_to_mass=True,
#                                                  slope=0.000001,
#                                                  weight=80.0)
# gemt.add_to_model()
# outputobjects.append(gemt)

# #--------------------------
# # Monte-Carlo Sampling
# #--------------------------

# #--------------------------
# # Set MC Sampling Parameters
# #--------------------------
# num_frames = 20000
# if '--test' in sys.argv: num_frames=100
# num_mc_steps = 10

In [17]:

# EM
em_map = Path(glob_data_dir, "em/maps/1031/1031_5_dsfact2_cutoff.05_ng100.txt")
em_weight = 1.0
sel = IMP.atom.Selection(
    hierarchy=root_hier,
    representation_type=IMP.atom.DENSITIES,
    copy_index=0
)

densities = sel.get_selected_particles()
gem = IMP.bayesianem.restraint.GaussianEMRestraintWrapper(
    densities,
    target_fn=str(em_map),
    scale_target_to_mass=True,
    # target_mass=25008.40380000003,
    slope=0.01,
    target_radii_scale=3.0,
    target_is_rigid_body=False,
    weight=em_weight
)
gem.add_to_model()
gem.set_label("BayesianEM")
output_objects.append(gem)

# ACTIVE SITE
# active_site_dr = IMP.pmi.restraints.basic.DistanceToPointRestraint(
#     root_hier=s.get_hierarchy(),
#     tuple_selection=(args.phos_res, args.phos_res, "AKT1", 0),
#     anchor_point=IMP.algebra.Vector3D(160.938, 122.210, 187.189),
#     radius=0,
#     kappa=1
# )
# active_site_dr.add_to_model()
# output_objects.append(active_site_dr)

# SHUFFLE
excluded_rbs = list()
excluded_rbs.extend(rbs["MTOR"])
excluded_rbs.extend(rbs["RICTOR"])
excluded_rbs.extend(rbs["MLST8"])
excluded_rbs.extend(rbs["MSIN1"][:2])

IMP.pmi.tools.shuffle_configuration(
    objects=root_hier,
    excluded_rigid_bodies=excluded_rbs,
    max_translation=100,
    max_rotation=2*math.pi,
    verbose=False,
    cutoff=10,
    niterations=25
)

will scale target mass by 1.0
will set target mass to 4540.044600000005 0.9737350235160739 4662.505189149192
target num particles 98 total weight 4540.044600000005
model num particles 40 total weight 4540.044600000005
done EM setup
shuffling 20 rigid bodies
shuffling 242 flexible beads


In [18]:
print("Start optimization")
dof.optimize_flexible_beads(100)

Start optimization
optimize_flexible_beads: optimizing 242 flexible beads


As it is generally undesirable to rerun the entire modeling just to generate an mmCIF file, we can save a lot of time when we do the replica exchange by turning on [ReplicaExchange](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1pmi_1_1macros_1_1ReplicaExchange.html)'s `test_mode`, which skips the actual Monte Carlo simulation (so we use the previously-generated trajectory):

In [19]:
print("Start sampling")
output_dir = Path("../data/0")
rex = IMP.pmi.macros.ReplicaExchange(
    m,
    root_hier=root_hier,
    monte_carlo_sample_objects=dof.get_movers(),
    global_output_directory=str(output_dir),
    output_objects=output_objects,
    monte_carlo_steps=20,
    number_of_best_scoring_models=10,
    number_of_frames=1,
    replica_exchange_maximum_temperature=5,
    test_mode=True
)

t0 = time.time()
rex.execute_macro()
print((time.time() - t0))

Start sampling
Setting up MonteCarlo
Setting up ReplicaExchange
ReplicaExchange: MPI was found. Using Parallel Replica Exchange
Setting up stat file
Stat file writing is disabled
ReplicaExchange: it generates initial.*.rmf3, stat.*.out, rmfs/*.rmf3 for each replica 
--- it stores the best scoring pdb models in pdbs/
--- the stat.*.out and rmfs/*.rmf3 are saved only at the lowest temperature
--- variables:
------ atomistic                      False
------ best_pdb_dir                   pdbs/
------ best_pdb_name_suffix           model
------ do_clean_first                 True
------ do_create_directories          True
------ geometries                     None
------ global_output_directory        ../data/0
------ initial_rmf_name_suffix        initial
------ mmcif                          False
------ molecular_dynamics_steps       10
------ monte_carlo_steps              20
------ monte_carlo_temperature        1.0
------ nframes_write_coordinates      1
------ num_sample_rounds    

In [20]:
# # This object defines all components to be sampled as well as the sampling protocol
# mc1=IMP.pmi.macros.ReplicaExchange(m,
#                                    root_hier=root_hier,
#                                    monte_carlo_sample_objects=dof.get_movers(),
#                                    output_objects=outputobjects,
#                                    monte_carlo_temperature=1.0,
#                                    simulated_annealing=True,
#                                    simulated_annealing_minimum_temperature=1.0,
#                                    simulated_annealing_maximum_temperature=2.5,
#                                    simulated_annealing_minimum_temperature_nframes=200,
#                                    simulated_annealing_maximum_temperature_nframes=20,
#                                    replica_exchange_minimum_temperature=1.0,
#                                    replica_exchange_maximum_temperature=2.5,
#                                    number_of_best_scoring_models=10,
#                                    monte_carlo_steps=num_mc_steps,
#                                    number_of_frames=num_frames,
#                                    global_output_directory="output",
#                                    test_mode=True)

# # Start Sampling
# mc1.execute_macro()

Once we're done with our PMI protocol, we call the [ProtocolOutput.finalize](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1pmi_1_1mmcif_1_1ProtocolOutput.html#a7b8395a33cbd3211cdf28e943c35dfa5) method to collect all of the information about the integrative modeling protocol in [ihm.System](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System):

In [21]:
po.finalize()

Note that the [ihm.System](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System) object, stored as the `system` attribute of [ProtocolOutput](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1pmi_1_1mmcif_1_1ProtocolOutput.html), contains a full description of the system:

In [22]:
s = po.system

print("all subunits:",
      [a.details for a in s.asym_units])

print("first subunit sequence:",
      "".join(r.code for r in s.asym_units[0].entity.sequence))

print("all restraints on the system:", s.restraints)

all subunits: ['MTOR.0', 'MTOR.1', 'RICTOR.0', 'RICTOR.1', 'MLST8.0', 'MLST8.1', 'MSIN1.0', 'MSIN1.1', 'AKT1.0', 'AKT1.1']
first subunit sequence: MLGTGPAAATTAATTSSNVSVLQQFASGLKSRNEETRAKAAKELQHYVTMELREMSQEESTRFYDQLNHHIFELVSSSDANERKGGILAIASLIGVEGGNATRIGRFANYLRNLLPSNDPVVMEMASKAIGRLAMAGDTFTAEYVEFEVKRALEWLGADRNEGRRHAAVLVLRELAISVPTFFFQQVQPFFDNIFVAVWDPKQAIREGAVAALRACLILTTQREPKEMQKPQWYRHTFEEAEKGFDETLAKEKGMNRDDRIHGALLILNELVRISSMEGERLREEMEEITQQQLVHDKYCKDLMGFGTKPRHITPFTSFQAVQPQQSNALVGLLGYSSHQGLMGFGTSPSPAKSTLVESRCCRDLMEEKFDQVCQWVLKCRNSKNSLIQMTILNLLPRLAAFRPSAFTDTQYLQDTMNHVLSCVKKEKERTAAFQALGLLSVAVRSEFKVYLPRVLDIIRAALPPKDFAHKRQKAMQVDATVFTCISMLARAMGPGIQQDIKELLEPMLAVGLSPALTAVLYDLSRQIPQLKKDIQDGLLKMLSLVLMHKPLRHPGMPKGLAHQLASPGLTTLPEASDVGSITLALRTLGSFEFEGHSLTQFVRHCADHFLNSEHKEIRMEAARTCSRLLTPSIHLISGHAHVVSQTAVQVVADVLSKLLVVGITDPDPDIRYCVLASLDERFDAHLAQAENLQALFVALNDQVFEIRELAICTVGRLSSMNPAFVMPFLRKMLIQILTELEHSGIGRIKEQSARMLGHLVSNAPRLIRPYMEPILKALILKLKDPDPDPNPGVINNVLATIGELAQVSGLEMRKWVDELFIIIMDMLQDSSLLAKRQVALWTLGQLVASTGY

Note that python-ihm only knows about three restraints, the two crosslinking restraints and the one EM density restraint, not the connectivity or excluded volume. This is because PDB considers that all *valid* structures satisfy connectivity and excluded volume by definition.

# mmCIF file format<a id="mmcif"></a>

At this point we have basic information which we can write out to an mmCIF file. Let's do that to get an idea of what the file looks like:

In [23]:
import ihm.dumper
with open('initial.cif', 'w') as fh:
    ihm.dumper.write(fh, [s])

mmCIF is a simple text format, so we can look at `initial.cif` in any text editor. Some simple data are stored in straightforward key:value pairs, e.g.

```
_struct.title 'Modeling of RNA Pol II'
```

More complex data can be stored as a table using a `loop` construct. This lists the fields (headings for the columns) after which the data (as rows) follow, e.g.

```
loop_
_software.pdbx_ordinal
_software.name
_software.classification
_software.description
_software.version
_software.type
_software.location
1 'IMP PMI module' 'integrative model building' 'integrative model building'
2.12.0 program https://integrativemodeling.org
2 'Integrative Modeling Platform (IMP)' 'integrative model building'
'integrative model building' 2.12.0 program https://integrativemodeling.org
```

Missing data are represented by `?` if they are unknown or `.` if they are deliberately omitted (for example if they don't make sense here).

In each case both the *categories* ([struct](http://mmcif.wwpdb.org/dictionaries/mmcif_pdbx_v50.dic/Categories/struct.html), [software](http://mmcif.wwpdb.org/dictionaries/mmcif_pdbx_v50.dic/Categories/software.html)) and the *data items* ([title](http://mmcif.wwpdb.org/dictionaries/mmcif_pdbx_v50.dic/Items/_struct.title.html), [pdbx_ordinal](http://mmcif.wwpdb.org/dictionaries/mmcif_pdbx_v50.dic/Items/_software.pdbx_ordinal.html), etc.) are defined by PDB, in the [PDBx dictionary](http://mmcif.wwpdb.org/dictionaries/mmcif_pdbx_v50.dic/Index/). This is the same dictionary used for regular PDB entries (e.g. crystal structures). Categories prefixed with `ihm` are in the [IHM dictionary](http://mmcif.wwpdb.org/dictionaries/mmcif_ihm.dic/Index/), which is specific to integrative models.

# Linking to other data<a id="linking"></a>

Integrative modeling draws on datasets from a variety of sources, so for a complete deposition all of this data need to be available. python-ihm provides an [ihm.dataset](https://python-ihm.readthedocs.io/en/latest/dataset.html#module-ihm.dataset) module to describe various types of dataset, such as input experimental data, auxiliary output files (such as localization densities), or workflow files (such as Python scripts for modeling or visualization). For example, each restraint has an associated dataset:

In [24]:
print("restraint datasets:", [r.dataset for r in s.restraints])

restraint datasets: [<ihm.dataset.CXMSDataset object at 0x35b767d40>, <ihm.dataset.CXMSDataset object at 0x35b767d40>, <ihm.dataset.EMDensityDataset object at 0x3464af740>]


The data are not placed directly in the mmCIF file - rather, the file contains links (using the [ihm.location](https://python-ihm.readthedocs.io/en/latest/location.html#module-ihm.location) module). These links can be:

 - an identifier in a domain-specific database, such as [PDB](https://www.wwpdb.org/) or [EMDB](https://www.ebi.ac.uk/pdbe/emdb/).
 - a [DOI](https://www.doi.org/) where the files can be obtained.
 - a path to a file on the local disk.

**Database identifiers** are preferable because the databases are curated by domain experts and include domain-specific information, and the files are in standard formats. ProtocolOutput will attempt to use these where possible. When a file is used for the modeling which cannot be tracked back to a database, ProtocolOutput will include its path (relative to that of the mmCIF file). For example, in this case the cross-links used are stored in simple CSV files and are linked as such:

In [25]:
# Datasets for XL-MS restraint
for r in s.restraints:
    if isinstance(r, ihm.restraint.CrossLinkRestraint):
        print("XL-MS dataset at:", r.dataset.location.path)
        print("Details:", r.dataset.location.details)

XL-MS dataset at: /Users/matthew/Documents/mtorc2/data/xlms/csvs/edc.csv
Details: Crosslinks
XL-MS dataset at: /Users/matthew/Documents/mtorc2/data/xlms/csvs/edc.csv
Details: Crosslinks


In addition, the Python script itself is linked from the mmCIF file. Such local paths won't be available to end users, so for deposition we need to replace these paths with database IDs or DOIs ([more on that later](#polishing)).

Datasets all support a simple hierarchy, where one dataset can be derived from another. In this case the EM restraint uses a locally-available GMM file, but it is derived from a density map which is stored in EMDB (as [EMD-1883](https://www.ebi.ac.uk/pdbe/entry/emdb/EMD-1883)). ProtocolOutput is able to read (using the [ihm.metadata module](https://python-ihm.readthedocs.io/en/latest/metadata.html#module-ihm.metadata)) metadata in both the GMM file and `.mrc` file to automatically determine this relationship:

In [26]:
# Dataset for EM restraint
em, = [r for r in s.restraints
       if isinstance(r, ihm.restraint.EM3DRestraint)]
d = em.dataset
print("GMM file at", d.location.path)
# print("is derived from EMDB entry", d.parents[0].location.access_code)

GMM file at /Users/matthew/Documents/mtorc2/data/em/maps/1031/1031_5_dsfact2_cutoff.05_ng100.txt


In cases where ProtocolOutput is not able to determine the parent of a dataset, we can create a dataset manually and add it to `d.parents`, either directly or using the [add_primary](https://python-ihm.readthedocs.io/en/latest/dataset.html#ihm.dataset.Dataset.add_primary) method. For example, see the [modeling of the NPC](https://github.com/integrativemodeling/npc2018/blob/main/template/1_modeling_wholeNPC.py#L1608-L1616) where we linked our EM restraint back to the parent (the density map in EMDB) and grandparent (cryo-ET tilt series in EMPIAR from which the density map was constructed).

As a further example of linkage, see the links in the previously-published [modeling of the Nup84 complex](https://salilab.org/nup84/) below. The mmCIF file links to the data directly used in the modeling (cross-links, crystal structures, electron microscopy class averages, comparative models, and Python scripts) via database IDs or DOIs. Furthermore, where available links are provided from this often-processed data back to the original data, such
as templates for comparative models, mass spectometry spectra for cross-links, or micrographs for class averages:

<img src="images/links.png" width="700px" title="Nup84 file linkage" />

# Annotation of input files<a id="annotation"></a>

ProtocolOutput will look at all input files to try to extract as much metadata
as possible. As [described above](#linking) this is used to look up database
identifiers, but it can also detect other inputs, such as the templates used
for comparative modeling. Thus, it is important for deposition that all input
files are annotated as well as possible:

 - deposit input files in a domain-specific database where possible and use
   the deposited file (which typically will contain identifying headers) in the
   modeling repository.
 - for PDB crystal structures, do not remove the original headers, such as
   the `HEADER` and `TITLE` lines.
 - for MODELLER comparative models, leave in the `REMARK` records and make sure
   that any files mentioned in `REMARK   6 ALIGNMENT:` or
   `REMARK   6 SCRIPT:` records are available (modify the paths if necessary,
   for example if you moved the PDB file into a different directory from the
   modeling script and alignment).
 - for manually generated PDB files, such as those extracted from a published
   work or generated by docking or other means, add suitable `EXPDTA` and
   `TITLE` records to the files for ProtocolOutput to pick up. See the
   [python-ihm docs](https://python-ihm.readthedocs.io/en/latest/metadata.html#ihm.metadata.PDBParser.parse_file)
   for more information.
 - for GMM files used for the EM density restraint, keep the original MRC
   file around and make sure that the `# data_fn:` header in the GMM file
   points to it.

# Polishing the deposition<a id="polishing"></a>

ProtocolOutput attempts to automatically generate as much as possible of the mmCIF file, but there are some areas where manual intervention is necessary because the data is missing, or its guess was incorrect. This data can be
corrected by manipulating the [ihm.System object](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System) directly, at the end of the script. We will look at a few examples in this section.

## Cross-linker type<a id="xltype"></a>

For cross-linking experiments, the mmCIF file contains a description of the cross-linking reagent used. This information is not in the CSV file. It can be provided in the Python script with the `linker` argument to passed to the PMI [CrossLinkingMassSpectrometryRestraint](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1pmi_1_1restraints_1_1crosslinking_1_1CrossLinkingMassSpectrometryRestraint.html), but if this argument was missing (or incorrect) we can correct it here. By checking the publications for the cross-link datasets from [Al Burlingame's lab](https://www.mcponline.org/content/13/2/420.long) and
[Juri Rappsilber's lab](http://emboj.embopress.org/content/29/4/717), we can determine that the [DSS](https://en.wikipedia.org/wiki/Disuccinimidyl_suberate) and [BS3](https://en.wikipedia.org/wiki/Bissulfosuccinimidyl_suberate) cross-linkers were used, respectively. These are common enough cross-linkers that python-ihm already includes definitions for them in the [ihm.cross_linkers](https://python-ihm.readthedocs.io/en/latest/cross_linkers.html#module-ihm.cross_linkers) module (for less common linkers we can create an [ihm.ChemDescriptor](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.ChemDescriptor) object from scratch to describe its chemistry). Then we just set the linker type for each [cross-linking restraint](https://python-ihm.readthedocs.io/en/latest/restraint.html#ihm.restraint.CrossLinkRestraint) in the [list of all restraints](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System.restraints):

In [27]:
# # Definitions of some common crosslinkers
# import ihm.cross_linkers

# # Set linker types to match those used (DSS and BS3)
# trnka, chen = [r for r in s.restraints
#                if isinstance(r, ihm.restraint.CrossLinkRestraint)]
# trnka.linker = ihm.cross_linkers.dss
# chen.linker = ihm.cross_linkers.bs3

## Correct number of output models<a id="fixnummodel"></a>

ProtocolOutput captures information on the actual sampling, storing it in [ihm.protocol.Step](https://python-ihm.readthedocs.io/en/latest/protocol.html#ihm.protocol.Step) objects. Here it correctly notes that we ran Monte Carlo to generate 20000 frames:

In [28]:
# Get last step of last protocol (protocol is an 'orphan' because
# we haven't used it for a model yet)
last_step = s.orphan_protocols[-1].steps[-1]
print(last_step.num_models_end)

1


However, in many modeling scenarios the modeling script is run multiple times on a compute cluster to generate several independent trajectories which are then combined. ProtocolOutput cannot know whether this happened. It is straightforward though to use the python-ihm API to manually change the number of output models to match that reported in the publication:

In [29]:
# Correct number of output models to account for multiple runs
# last_step.num_models_end = 200000

## Add UniProt sequence information<a id="uniprot"></a>

Usually the sequences for each subunit we modeled are available in a reference database such as
[UniProt](https://www.uniprot.org/). IMP doesn't need to know the database accession codes in order
to perform the modeling, but it is useful to link them for the deposition. We can do this using the
python-ihm API to add [ihm.reference.UniProtSequence](https://python-ihm.readthedocs.io/en/latest/reference.html#ihm.reference.UniProtSequence) objects. These are added per *entity*, not
per subunit (an entity has a unique sequence; if multiple subunits or copies have the same sequence,
they all map to the same entity). ProtocolOutput provides an `entities` dict, which maps our subunit
names (without copy numbers) to [ihm.Entity](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.Entity) objects:

In [30]:
# for subunit, accession in [('Rpb1', 'P04050'),
#                            ('Rpb2', 'P08518')]:
#     ref = ihm.reference.UniProtSequence.from_accession(accession)
#     po.entities[subunit].references.append(ref)

Here, [from_accession](https://python-ihm.readthedocs.io/en/latest/reference.html#ihm.reference.UniProtSequence.from_accession) queries the UniProt API to get full information
(so requires a network connection). Alternatively, we could create [ihm.reference.UniProtSequence](https://python-ihm.readthedocs.io/en/latest/reference.html#ihm.reference.UniProtSequence)
objects outselves. Here we just populate the first two sequences for illustration.

Sometimes the sequence modeled by IMP differs from that in UniProt. In this case the alignment between the two and/or any single-point mutations must be annotated with [ihm.reference.Alignment](https://python-ihm.readthedocs.io/en/latest/reference.html#ihm.reference.Alignment) and [ihm.reference.SeqDif](https://python-ihm.readthedocs.io/en/latest/reference.html#ihm.reference.SeqDif) objects:

 - For example, in our modeling of the NPC we only modeled the C-terminal domain of the Nup145 subunit. Thus, the sequence of the Nup145c Entity actually starts at residue 606 in the [UniProt sequence](https://www.uniprot.org/uniprotkb/P49687/entry). See the [NPC scripts](https://github.com/integrativemodeling/npc2018/blob/1299051cf83c3b60514b2f7c1780c60814a3f292/template/1_modeling_wholeNPC.py#L1849-L1868) for an example of creating a suitable Alignment object specifying the offset `db_begin`.
 - See the [pol_ii_g scripts](https://github.com/integrativemodeling/pol_ii_g/blob/a416964fe024352352789d1be8fbd7cfd288832f/production_scripts/sample.py#L288-L305)
for an example where the modeled sequence of the RPB5.0 subunit contains four single-point mutations compared to the UniProt sequence P19388.

## Add model coordinates<a id="addcoords"></a>

The current mmCIF file contains all of the input data, and notes that Monte Carlo was used to generate frames, but doesn't actually store any of those coordinates in the file. Ultimately this information will be added automatically
by PMI model analysis and validation scripts, but for the time being any information about clustering, localization densities, and final models needs to be added to the file using the python-ihm API:

In [31]:
# Get last protocol in the file
protocol = po.system.orphan_protocols[-1]
# State that we filtered the 200000 frames down to one cluster of
# 100 models:
analysis = ihm.analysis.Analysis()
protocol.analyses.append(analysis)
analysis.steps.append(ihm.analysis.ClusterStep(
                      feature='RMSD', num_models_begin=200000,
                      num_models_end=100))

python-ihm allows for models to be placed into groups ([ihm.model.ModelGroup](https://python-ihm.readthedocs.io/en/latest/model.html#ihm.model.ModelGroup)) so we can make such a group for the cluster. These groups are in turn grouped in an [ihm.model.State](https://python-ihm.readthedocs.io/en/latest/model.html#ihm.model.State), a distinct state of the system (e.g. open/closed form, or reactant/product in a chemical reaction). Finally, states can themselves be grouped into [ihm.model.StateGroup](https://python-ihm.readthedocs.io/en/latest/model.html#ihm.model.StateGroup) objects. All of these groups act like regular Python lists. In this case we have only one state, so we just add our model group to that:

In [32]:
mg = ihm.model.ModelGroup(name="Cluster 0")

# Add to last state
po.system.state_groups[-1][-1].append(mg)

Next, we can describe the cluster using the [ihm.model.Ensemble](https://python-ihm.readthedocs.io/en/latest/model.html#ihm.model.Ensemble) class, noting that it was generated from our clustering:

In [33]:
e = ihm.model.Ensemble(model_group=mg,
                       num_models=1,
                       post_process=analysis.steps[-1],
                       name="Cluster 0")
po.system.ensembles.append(e)

To add a model to the file, we create an [ihm.model.Model](https://python-ihm.readthedocs.io/en/latest/model.html#ihm.model.Model) object, add atoms or coarse-grained objects to it, and then add that to the previously-created [ModelGroup](https://python-ihm.readthedocs.io/en/latest/model.html#ihm.model.ModelGroup). ProtocolOutput provides a convenience function `add_model` which does this, converting the current IMP structure to python-ihm (we just need to load an IMP structure first, e.g. from an RMF file using [IMP.rmf.load_frame](https://integrativemodeling.org/2.23.0/doc/ref/namespaceIMP_1_1rmf.html#abc15209a0aef0e859c99ce00659ebdfb)).

In [34]:
import RMF
import IMP.rmf

# Add the model from RMF (let's say it's frame 42 from the output RMF file)
rmf_file = Path(glob_data_dir, "models/submission_3/136_2_1/cluster.0/cluster_center_model.rmf3")
rh = RMF.open_rmf_file_read_only(str(rmf_file))
IMP.rmf.link_hierarchies(rh, [root_hier])
IMP.rmf.load_frame(rh, RMF.FrameID(0))
del rh
model = po.add_model(e.model_group)

In practice we typically want to store more than just a single model in the file, since a single model cannot represent the flexibility and variability of the complete ensemble. mmCIF files allow for storing multiple conformations (like a traditional PDB file for storing an NMR ensemble) and also representations of the entire ensemble (localization densities).

However, mmCIF is a text format and is not best suited for storing large numbers of structures. We can instead store the coordinates in a simpler, binary file and link to it from the mmCIF using the [ihm.location.OutputFileLocation](https://python-ihm.readthedocs.io/en/latest/location.html#ihm.location.OutputFileLocation) class, providing one or more representative structures (such as the cluster centroid) in the mmCIF file itself. [DCD](https://www.ks.uiuc.edu/Research/namd/2.9/ug/node11.html) is one such format that we can (ab)use for this purpose (it is really designed for atomic trajectories). python-ihm provides an [ihm.model.DCDWriter](https://python-ihm.readthedocs.io/en/latest/model.html#ihm.model.DCDWriter) class which, given an [ihm.model.Model](https://python-ihm.readthedocs.io/en/latest/model.html#ihm.model.Model), will output DCD.

If we have localization densities from the analysis for the cluster, we can add those to the Ensemble as [ihm.model.LocalizationDensity](https://python-ihm.readthedocs.io/en/latest/model.html#ihm.model.LocalizationDensity) objects. These take python-ihm subunits (or subsets of them) as [ihm.AsymUnit](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.AsymUnit) objects. ProtocolOutput provides an `asym_units` member, which is a Python dict, to get python-ihm subunits given PMI component names (such as "Rpb4.0"). For example, we can easily add the localization density of the Rpb4 subunit for the entire cluster to the file:

In [35]:
# # Look up the ihm.AsymUnit corresponding to a PMI component name
# asym = po.asym_units['Rpb4.0']
# # Add path to a local output file
# loc = ihm.location.OutputFileLocation('output/cluster0.Rpb4.mrc')
# den = ihm.model.LocalizationDensity(file=loc, asym_unit=asym)
# # Add to ensemble
# e.densities.append(den)

In [42]:
for prot_name in ["MTOR", "RICTOR", "MLST8", "MSIN1", "AKT1"]:
    asym = po.asym_units["{}.0".format(prot_name)]
    density_file = Path(glob_data_dir, "models/submission_3/136_2_1/cluster.0/LPD_{}.mrc".format(prot_name))

    loc = ihm.location.OutputFileLocation(str(density_file))
    den = ihm.model.LocalizationDensity(file=loc, asym_unit=asym)
    e.densities.append(den)


(We can also add information about the fit of the model to each of the restraints. Most python-ihm restraint objects contain a `fits` member which is a Python dict with [ihm.model.Model](https://python-ihm.readthedocs.io/en/latest/model.html#ihm.model.Model) objects as keys and some sort of restraint-fit object as values. For example, we could note that the cluster centroid model `m` was fit against the EM map by adding an [ihm.restraint.EM3DRestraintFit](https://python-ihm.readthedocs.io/en/latest/restraint.html#ihm.restraint.EM3DRestraintFit) object with the  cross correlation coefficient, if known. We could also specify the Bayesian nuisances ψ and σ for our cross-links with [ihm.restraint.CrossLinkFit](https://python-ihm.readthedocs.io/en/latest/restraint.html#ihm.restraint.CrossLinkFit) objects.)

## Replace local links with DOIs<a id="adddois"></a>

We added paths for localization densities and the DCD file with [ihm.location.FileLocation](https://python-ihm.readthedocs.io/en/latest/location.html#ihm.location.FileLocation) objects. These files won't be accessible to end users since they live on the local disk. ([Recall from earlier](#linking) that ProtocolOutput also adds local paths for the cross-link CSV files.)

`FileLocation` takes an optional `repo` argument which, if given, is an [ihm.location.Repository](https://python-ihm.readthedocs.io/en/latest/location.html#ihm.location.Repository) describing the DOI where the files can be found, generally as a `zip` file (the first argument to [ihm.location.FileLocation](https://python-ihm.readthedocs.io/en/latest/location.html#ihm.location.FileLocation) is a path within the repository if `repo` is specified, otherwise a path on the local disk). So we could explicitly refer to a DOI for each of our external files. (A number of services provide DOIs for uploaded files, such as [Zenodo](https://zenodo.org/) or [FigShare](https://figshare.com/).)

An alternative is to retroactively change the existing `FileLocation`s (both those created automatically by ProtocolOutput and manually by us using python-ihm) using the `root` and `top_directory` arguments to [Repository](https://python-ihm.readthedocs.io/en/latest/location.html#ihm.location.Repository) and the [ihm.System.update_locations_in_repositories](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System.update_locations_in_repositories) function:

In [43]:
# # Replace local links with DOIs
# repo = ihm.location.Repository(doi="10.5281/zenodo.2598760", root="../..",
#                   top_directory="salilab-imp_deposition_tutorial-1ad5919",
#                   url="https://zenodo.org/record/2598760/files/salilab/"
#                       "imp_deposition_tutorial-v0.2.zip")
# s.update_locations_in_repositories([repo])

This assumes that the zipfile at the given DOI and URL was created by archiving all files under the top directory of the [GitHub repository](https://github.com/salilab/imp_deposition_tutorial) (`../..` relative to the directory containing the script). (It was actually created automatically [by Zenodo](https://zenodo.org) when a new release of this tutorial was created on GitHub. Zenodo puts all the files in the zipfile under a uniquely-named directory, so we need to set `top_directory` to match.)

> In principle archiving a GitHub repository at Zenodo is as simple as downloading the repository from GitHub as a zip file, then uploading that file to Zenodo. In practice, however, these files might be quite large, and so viewers such as ChimeraX may need to download a multi-gigabyte zip file just to get a relatively small input, such as a CSV or FASTA file. To remedy this, the repository can be split into multiple smaller zip files. See for example the multiple files archiving the [2018 NPC study](https://doi.org/10.5281/zenodo.1194547); these were generated using [this script](https://github.com/integrativemodeling/npc2018/blob/main/util/make_archive.py).

Thus the cross-link file which was previously linked with the local path `../data/polii_xlinks.csv` can now be found by downloading `imp_deposition_tutorial-v0.2.zip` and extracting `salilab-imp_deposition_tutorial-1ad5919/rnapolii/data/polii_xlinks.csv` from it:

In [44]:
# # Datasets for XL-MS restraint
# trnka, chen = [r for r in s.restraints
#                if isinstance(r, ihm.restraint.CrossLinkRestraint)]
# d = trnka.dataset
# print("XL-MS dataset now at %s/%s inside %s"
#       % (d.location.repo.top_directory,
#          d.location.path, d.location.repo.url))

# Output<a id="output"></a>

We can now save the entire protocol to an mmCIF file (or to [BinaryCIF](https://github.com/dsehnal/BinaryCIF), if we have the Python [msgpack](https://pypi.org/project/msgpack/) package installed) using the [ihm.dumper.write](https://python-ihm.readthedocs.io/en/latest/dumper.html#ihm.dumper.write) function:

In [45]:
po.finalize()
with open('../data/mtorc2_akt.cif', 'w') as fh:
    ihm.dumper.write(fh, [po.system])

#with open('rnapoliii.bcif', 'wb') as fh:
#    ihm.dumper.write(fh, [s], format='BCIF')

(BinaryCIF stores all the same data as mmCIF, but rather than storing it as a text file, the data is compressed and then written out as a [MessagePack](https://msgpack.org/index.html) file, which is a form of binary JSON.)

# Visualization<a id="visualization"></a>

## ChimeraX<a id="chimerax"></a>

mmCIF files can be viewed in many viewers. However, most viewers do not yet support the integrative modeling extensions, and so may only show the atomic parts of the model (if any). Integrative models can be viewed in
[ChimeraX](https://www.rbvi.ucsf.edu/chimerax/) - be sure to use a recent nightly build, and open the file from the ChimeraX command line using the `format ihm` option, e.g. `open rnapolii.cif format ihm`. (If you also want to see any DCD files, add `ensembles true` to the end of your command line to load them and then use the ChimeraX `coordset` command to manipulate the set of coordinates, e.g. `coordset slider #1.3.2`.)

ChimeraX also supports downloading and displaying structures directly from the precursor [PDB-Dev](https://pdb-dev.wwpdb.org/) database, for example from the ChimeraX command line `open 10 from pdbdev`.

> Note that even though PDB-IHM is still quite small, the models vary widely in composition, e.g.
>
> - Some models (e.g. [8zzi](https://pdb-ihm.org/entry.html?8ZZI)) are atomic and look much like a traditional "PDB" x-ray structure.
> - Some models (e.g. [8zz2](https://pdb-ihm.org/entry.html?8ZZ2)) contain multiple states that have different sequences (this particular case contains exosome in nucleus-localized and cytoplasm-localized forms).
> - Some models are not of proteins at all - e.g. [8zz8](https://pdb-ihm.org/entry.html?8ZZ8) is a model of chromatin, part of the fly genome.
> - Some models contain multiple representations - e.g. [8zzc](https://pdb-ihm.org/entry.html?8ZZC), a model of the yeast nuclear pore complex, contains both models of the scaffold (ring) and the flexible FG repeat regions which occupy the center of the pore.

[VMD](http://www.ks.uiuc.edu/Research/vmd/) is also reportedly working on support in their forthcoming 1.9.4 release.

An example view of the deposition, as rendered by ChimeraX, is shown below. The EM map is shown as a mesh, the DSS cross-links as green dashed lines, and the BS3 cross-links as blue dashed lines. ChimeraX also shows the comparative models used as starting guesses for the subunit structures - toggle off the display of the result models to see them.

<img src="images/chimerax.png" width="700px" title="ChimeraX visualization of rnapolii mmCIF file" />

## Web browser<a id="browser"></a>

mmCIF files can also be viewed in a web browser, using [Mol* Viewer](https://molstar.org/viewer). (Note that it currently only shows the coordinates, not the experimental information or other metadata.)

## Plain text<a id="plain"></a>

mmCIF files are also just plain text files, and can be viewed in any text editor (for example to check for errors, particularly for categories that ChimeraX doesn't support yet). Most of the data is stored as simple tables
(look for the `loop_` keyword in the file). For example, the coordinates of the coarse-grained beads are stored in the [_ihm_sphere_obj_site table](http://mmcif.wwpdb.org/dictionaries/mmcif_ihm.dic/Categories/ihm_sphere_obj_site.html),
the start of which in `rnapolii.cif` looks like:

```
loop_
_ihm_sphere_obj_site.ordinal_id
_ihm_sphere_obj_site.entity_id
_ihm_sphere_obj_site.seq_id_begin
_ihm_sphere_obj_site.seq_id_end
_ihm_sphere_obj_site.asym_id
_ihm_sphere_obj_site.Cartn_x
_ihm_sphere_obj_site.Cartn_y
_ihm_sphere_obj_site.Cartn_z
_ihm_sphere_obj_site.object_radius
_ihm_sphere_obj_site.rmsf
_ihm_sphere_obj_site.model_id
1 1 1 1 A 158.180 180.876 96.861 3.068 . 1
2 1 2 2 A 105.913 135.689 154.058 2.888 . 1
3 1 3 3 A 104.305 132.810 156.027 2.273 . 1
4 1 4 4 A 101.027 134.743 156.557 3.008 . 1
```

This simply states that a sphere representing residue 1 in chain A is centered
at (158.180, 180.876, 96.861) and has radius 3.068, and so on.

## As Python objects<a id="python"></a>

We can also read in mmCIF or BinaryCIF files using python-ihm's [ihm.reader](https://python-ihm.readthedocs.io/en/latest/reader.html#module-ihm.reader) module and explore the data using the python-ihm API. For example, we can read the structure we just created:

In [ ]:
import ihm.reader
with open('rnapolii.cif') as fh:
    s, = ihm.reader.read(fh)
print(s.title, s.restraints, s.ensembles, s.state_groups)

Similarly we could explore any integrative model deposited in PDB-IHM. For example we can look at PDB entry 8ZZE, a [HADDOCK](http://www.bonvinlab.org/software/haddock2.2/) model:

In [ ]:
import urllib.request

fh = urllib.request.urlopen('https://pdb-ihm.org/cif/8zze.cif')
s, = ihm.reader.read(fh)
print(s.title, s.restraints, s.ensembles, s.state_groups)

# Further reading<a id="further"></a>

More tutorials on using IMP are available at [the IMP web site](https://integrativemodeling.org/tutorials/).